## Initial Testing

In [3]:
import kinematics as kin
from visualization import VizScene
from pyqtgraph.Qt import QtGui
import time
import numpy as np
import mss
import cv2
np.set_printoptions(precision=4, suppress=True)

#dh order: theta, d, a, alpha
dh_dlr = [[0,0,0,-np.pi/2],
          [0,0,0,np.pi/2],
          [0,1,0,-np.pi/2],
          [0,0,0,np.pi/2],
          [0,1,0,-np.pi/2],
          [0,0,0,np.pi/2],
          [0,1,0,0]]

arm = kin.SerialArm(dh_dlr)

q_set_0 = [0,0,0,0,0,0,0]
# q_set_0 = [np.pi/3,np.pi/3,np.pi/3,np.pi/3,np.pi/3,np.pi/3,np.pi/3]
T = arm.fk(q_set_0)
# pos = T[0:3,3].T
pos = [0,0,0]
# print(pos)

viz = VizScene()
viz.add_arm(arm, draw_frames=True)
viz.update(qs=[q_set_0])
viz.add_box([.2,.2,.2],[1,0,1])
viz.window.setCameraPosition(pos=QtGui.QVector3D(pos[0],pos[1],pos[2]),
                             distance=10,elevation=0,azimuth=90)
viz.hold()

with mss.mss() as sct:
    screen_w, screen_h = 1920, 1200
    fraction_of_screen = 0.7 # could be a function parameter
    x = int(screen_w * (1-fraction_of_screen)) // 2
    y = int(screen_h * (1-fraction_of_screen)) // 2
    win_w = int(screen_w * fraction_of_screen)
    win_h = int(screen_h * fraction_of_screen)
    # monitor = sct.monitors[1]
    monitor_region = {"top": y, "left": x, "width": win_w, "height": win_h}
    sct_image = sct.grab(monitor_region)
    scsh = np.array(sct_image)
    scsh1 = cv2.cvtColor(scsh, cv2.COLOR_RGB2BGR) #colored image
    scsh = cv2.cvtColor(scsh, cv2.COLOR_RGB2HSV) #image for filtering
    scgrey = cv2.cvtColor(scsh, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(scgrey, 127, 255, cv2.THRESH_BINARY)

    L_limit = np.array([80,50,50])
    U_limit = np.array([100,255,255])
    b_mask = cv2.inRange(scsh,L_limit,U_limit) #filter image
    blue = cv2.bitwise_and(scsh1, scsh1,mask=b_mask) #make blue mask
    scgrey = cv2.cvtColor(blue, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(scgrey, 20, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    print(contours)

    # cv2.imshow("Screen Capture", scsh1)
    # cv2.waitKey(0)
    cv2.drawContours(scgrey, contours, -1, (255,0,0), 5)

    cv2.imshow("Screen Capture", scgrey)
    # cv2.drawContours(scgrey,contours,-1,(0,255,0),2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# print(viz.window.items)

(array([[[543, 291]],

       [[543, 314]],

       [[544, 315]],

       [[567, 315]],

       [[567, 292]],

       [[566, 291]]], dtype=int32), array([[[672,  67]]], dtype=int32), array([[[669,  67]]], dtype=int32), array([[[668,  60]]], dtype=int32))


## Single Frame Centroid Finding in 3D

In [1]:
import kinematics as kin
from visualization import VizScene
from pyqtgraph.Qt import QtGui
import time
import numpy as np
import mss
import cv2
np.set_printoptions(precision=4, suppress=True)


#dh order: theta, d, a, alpha
dh_dlr = [[0,0,0,-np.pi/2],
          [0,0,0,np.pi/2],
          [0,1,0,-np.pi/2],
          [0,0,0,np.pi/2],
          [0,1,0,-np.pi/2],
          [0,0,0,np.pi/2],
          [0,1,0,0]]

arm = kin.SerialArm(dh_dlr)

q_set_0 = [0,0,0,0,0,0,0]
# q_set_0 = [np.pi/3,np.pi/3,np.pi/3,np.pi/3,np.pi/3,np.pi/3,np.pi/3]
T = arm.fk(q_set_0)
# pos = T[0:3,3].T
pos = [0,0,0]
# print(pos)

obs_pos = [-1,1,2]
obj_pos = [2,2,2]
if(np.linalg.norm(obj_pos) > 3):
    print("Objective out of workspace")

viz = VizScene()
viz.add_arm(arm, draw_frames=False)
viz.update(qs=[q_set_0])
viz.add_box([.2,.2,.2],obs_pos) #obstacle
viz.add_box([.2,.2,.2],obj_pos,bcolor=np.array([0, 0.7, 0, 1])) #objective
viz.window.setCameraPosition(pos=QtGui.QVector3D(pos[0],pos[1],pos[2]),
                             distance=10,elevation=0,azimuth=90)
viz.hold()

with mss.mss() as sct:
    screen_w, screen_h = 1920, 1200
    fraction_of_screen = 0.7 # could be a function parameter
    x = int(screen_w * (1-fraction_of_screen)) // 2
    y = int(screen_h * (1-fraction_of_screen)) // 2
    win_w = int(screen_w * fraction_of_screen)
    win_h = int(screen_h * fraction_of_screen)
    midscreen_w = win_w/2
    midscreen_h = win_h/2
    # monitor = sct.monitors[1]
    monitor_region = {"top": y, "left": x, "width": win_w, "height": win_h}
    scsh = sct.grab(monitor_region)
    sct_image = np.array(scsh)
    # gray_image = cv2.cvtColor(sct_image, cv2.COLOR_BGR2GRAY)
    hsv_image = cv2.cvtColor(sct_image, cv2.COLOR_RGB2HSV)

    obstacle_L_limit = np.array([80,50,50])
    obstacle_U_limit = np.array([100,255,255])
    obstacle_mask = cv2.inRange(hsv_image, obstacle_L_limit, obstacle_U_limit)
    obstacle_image = cv2.bitwise_and(hsv_image, hsv_image, mask=obstacle_mask)
    obstacle_grey = cv2.cvtColor(obstacle_image, cv2.COLOR_BGR2GRAY)
    obs_ret, obs_thresh = cv2.threshold(obstacle_grey, 20, 255, cv2.THRESH_BINARY)
    obs_contours, obs_hierarchy = cv2.findContours(obs_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    obs_x1 = -1*(np.average(obs_contours[0][:,:,0]) - midscreen_w)/117
    obs_z1 = -1*(np.average(obs_contours[0][:,:,1]) - midscreen_h)/117
    # print("Obstacle Centroid 1")
    # print("Obs X: ", obs_x1)
    # print("Obs Z: ", obs_z1)

    objective_L_limit = np.array([55,50,50])
    objective_U_limit = np.array([60,255,255])
    objective_mask = cv2.inRange(hsv_image, objective_L_limit, objective_U_limit)
    objective_image = cv2.bitwise_and(hsv_image, hsv_image, mask=objective_mask)   
    objective_grey = cv2.cvtColor(objective_image, cv2.COLOR_BGR2GRAY)
    obj_ret, obj_thresh = cv2.threshold(objective_grey, 20, 255, cv2.THRESH_BINARY)
    obj_contours, obj_hierarchy = cv2.findContours(obj_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # print("Objective Contours", obj_contours[0])
    obj_x1 = -1*(np.average(obj_contours[0][:,:,0]) - midscreen_w)/117
    obj_z1 = -1*(np.average(obj_contours[0][:,:,1]) - midscreen_h)/117
    # print("Obstacle Centroid 1")
    # print("Obs X: ", obj_x1)
    # print("Obs Z: ", obj_z1)

    # cv2.imshow("Screen", obstacle_image)
    # cv2.waitKey(0)
    # cv2.imshow("Screen", hsv_image)
    # cv2.waitKey(0)
    cv2.destroyAllWindows()

viz = VizScene()
viz.add_arm(arm, draw_frames=False)
viz.update(qs=[q_set_0])
viz.add_box([.2,.2,.2],obs_pos) #obstacle
viz.add_box([.2,.2,.2],obj_pos,bcolor=np.array([0, 0.7, 0, 1])) #objective
viz.window.setCameraPosition(pos=QtGui.QVector3D(pos[0],pos[1],pos[2]),
                             distance=10,elevation=0,azimuth=0)
viz.hold()

with mss.mss() as sct:
    screen_w, screen_h = 1920, 1200
    fraction_of_screen = 0.7 # could be a function parameter
    x = int(screen_w * (1-fraction_of_screen)) // 2
    y = int(screen_h * (1-fraction_of_screen)) // 2
    win_w = int(screen_w * fraction_of_screen)
    win_h = int(screen_h * fraction_of_screen)
    midscreen_w = win_w/2
    midscreen_h = win_h/2
    # monitor = sct.monitors[1]
    monitor_region = {"top": y, "left": x, "width": win_w, "height": win_h}
    scsh = sct.grab(monitor_region)
    sct_image = np.array(scsh)
    # gray_image = cv2.cvtColor(sct_image, cv2.COLOR_BGR2GRAY)
    hsv_image = cv2.cvtColor(sct_image, cv2.COLOR_RGB2HSV)

    obstacle_L_limit = np.array([80,50,50])
    obstacle_U_limit = np.array([100,255,255])
    obstacle_mask = cv2.inRange(hsv_image, obstacle_L_limit, obstacle_U_limit)
    obstacle_image = cv2.bitwise_and(hsv_image, hsv_image, mask=obstacle_mask)
    obstacle_grey = cv2.cvtColor(obstacle_image, cv2.COLOR_BGR2GRAY)
    obs_ret, obs_thresh = cv2.threshold(obstacle_grey, 20, 255, cv2.THRESH_BINARY)
    obs_contours, obs_hierarchy = cv2.findContours(obs_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    obs_y2 = (np.average(obs_contours[0][:,:,0]) - midscreen_w)/117
    obs_z2 = -1*(np.average(obs_contours[0][:,:,1]) - midscreen_h)/117
    # print("Obstacle Centroid 2")
    # print("Obs Y: ", obs_y2)
    # print("Obs Z: ", obs_z2)

    objective_L_limit = np.array([55,50,50])
    objective_U_limit = np.array([60,255,255])
    objective_mask = cv2.inRange(hsv_image, objective_L_limit, objective_U_limit)
    objective_image = cv2.bitwise_and(hsv_image, hsv_image, mask=objective_mask)   
    objective_grey = cv2.cvtColor(objective_image, cv2.COLOR_BGR2GRAY)
    obj_ret, obj_thresh = cv2.threshold(objective_grey, 20, 255, cv2.THRESH_BINARY)
    obj_contours, obj_hierarchy = cv2.findContours(obj_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # print("Objective Contours", obj_contours[0])
    obj_y2 = (np.average(obj_contours[0][:,:,0]) - midscreen_w)/117
    obj_z2 = -1*(np.average(obj_contours[0][:,:,1]) - midscreen_h)/117
    # print("Obstacle Centroid 2")
    # print("Obs Y: ", obj_y2)
    # print("Obs Z: ", obj_z2)

    # cv2.imshow("Screen", obstacle_image)
    # cv2.waitKey(0)
    # cv2.imshow("Screen", hsv_image)
    # cv2.waitKey(0)
    cv2.destroyAllWindows()



viz = VizScene()
viz.add_arm(arm, draw_frames=False)
viz.update(qs=[q_set_0])
viz.add_box([.2,.2,.2],obs_pos) #obstacle
viz.add_box([.2,.2,.2],obj_pos,bcolor=np.array([0, 0.7, 0, 1])) #objective
viz.window.setCameraPosition(pos=QtGui.QVector3D(pos[0],pos[1],pos[2]),
                             distance=10,elevation=90,azimuth=0)
viz.hold()

with mss.mss() as sct:
    screen_w, screen_h = 1920, 1200
    fraction_of_screen = 0.7 # could be a function parameter
    x = int(screen_w * (1-fraction_of_screen)) // 2
    y = int(screen_h * (1-fraction_of_screen)) // 2
    win_w = int(screen_w * fraction_of_screen)
    win_h = int(screen_h * fraction_of_screen)
    midscreen_w = win_w/2
    midscreen_h = win_h/2
    # monitor = sct.monitors[1]
    monitor_region = {"top": y, "left": x, "width": win_w, "height": win_h}
    scsh = sct.grab(monitor_region)
    sct_image = np.array(scsh)
    # gray_image = cv2.cvtColor(sct_image, cv2.COLOR_BGR2GRAY)
    hsv_image = cv2.cvtColor(sct_image, cv2.COLOR_RGB2HSV)

    obstacle_L_limit = np.array([80,50,50])
    obstacle_U_limit = np.array([100,255,255])
    obstacle_mask = cv2.inRange(hsv_image, obstacle_L_limit, obstacle_U_limit)
    obstacle_image = cv2.bitwise_and(hsv_image, hsv_image, mask=obstacle_mask)
    obstacle_grey = cv2.cvtColor(obstacle_image, cv2.COLOR_BGR2GRAY)
    obs_ret, obs_thresh = cv2.threshold(obstacle_grey, 20, 255, cv2.THRESH_BINARY)
    obs_contours, obs_hierarchy = cv2.findContours(obs_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    obs_y3 = (np.average(obs_contours[0][:,:,0]) - midscreen_w)/117
    obs_x3 = (np.average(obs_contours[0][:,:,1]) - midscreen_h)/117
    # print("Obstacle Centroid 3")
    # print("Obs X: ", obs_x3)
    # print("Obs Y: ", obs_y3)

    objective_L_limit = np.array([55,50,50])
    objective_U_limit = np.array([60,255,255])
    objective_mask = cv2.inRange(hsv_image, objective_L_limit, objective_U_limit)
    objective_image = cv2.bitwise_and(hsv_image, hsv_image, mask=objective_mask)   
    objective_grey = cv2.cvtColor(objective_image, cv2.COLOR_BGR2GRAY)
    obj_ret, obj_thresh = cv2.threshold(objective_grey, 20, 255, cv2.THRESH_BINARY)
    obj_contours, obj_hierarchy = cv2.findContours(obj_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # print("Objective Contours", obj_contours[0])
    obj_y3 = (np.average(obj_contours[0][:,:,0]) - midscreen_w)/117
    obj_x3 = (np.average(obj_contours[0][:,:,1]) - midscreen_h)/117
    # print("Obstacle Centroid 3")
    # print("Obs X: ", obj_x3)
    # print("Obs Y: ", obj_y3)

    # cv2.imshow("Screen", obstacle_image)
    # cv2.waitKey(0)
    # cv2.imshow("Screen", hsv_image)
    # cv2.waitKey(0)
    cv2.destroyAllWindows()

print("Obstacle Position: ", str(obs_pos))
print("Obstacle X 1: ", obs_x1)
print("Obstacle X 3: ", obs_x3)
print("Obstacle Y 2: ", obs_y2)
print("Obstacle Y 3: ", obs_y3)
print("Obstacle Z 1: ", obs_z1)
print("Obstacle Z 3: ", obs_z2)

print("Objective Position: ", str(obj_pos))
print("Objective X 1: ", obj_x1)
print("Objective X 3: ", obj_x3)
print("Objective Y 2: ", obj_y2)
print("Objective Y 3: ", obj_y3)
print("Objective Z 1: ", obj_z1)
print("Objective Z 2: ", obj_z2)

Objective out of workspace
Obstacle Position:  [-1, 1, 2]
Obstacle X 1:  -1.1025641025641026
Obstacle X 3:  -1.2478632478632479
Obstacle Y 2:  0.8760683760683761
Obstacle Y 3:  1.2393162393162394
Obstacle Z 1:  2.217948717948718
Obstacle Z 3:  1.8311965811965811
Objective Position:  [2, 2, 2]
Objective X 1:  2.486324786324786
Objective X 3:  2.4777777777777774
Objective Y 2:  2.4777777777777774
Objective Y 3:  2.4777777777777774
Objective Z 1:  2.486324786324786
Objective Z 2:  2.486324786324786


## Function Definitions for Simplified Implementation

In [1]:
import kinematics as kin
from visualization import VizScene
from pyqtgraph.Qt import QtGui
import time
import numpy as np
import mss
import cv2
np.set_printoptions(precision=4, suppress=True)


#dh order: theta, d, a, alpha
dh_dlr = [[0,0,0,-np.pi/2],
          [0,0,0,np.pi/2],
          [0,1,0,-np.pi/2],
          [0,0,0,np.pi/2],
          [0,1,0,-np.pi/2],
          [0,0,0,np.pi/2],
          [0,1,0,0]]

arm = kin.SerialArm(dh_dlr)

q_set_0 = [0,0,0,0,0,0,0]
# q_set_0 = [np.pi/3,np.pi/3,np.pi/3,np.pi/3,np.pi/3,np.pi/3,np.pi/3]
T = arm.fk(q_set_0)
# pos = T[0:3,3].T
pos = [0,0,0]
# print(pos)


obs_pos = [-1,1,2]
obj_pos = [1,1,2]
if(np.linalg.norm(obj_pos) > 3):
    print("Objective out of workspace")

viz = VizScene()
viz.add_arm(arm, draw_frames=False)
viz.update(qs=[q_set_0])
viz.add_box([.2,.2,.2],obs_pos) #obstacle
viz.add_box([.2,.2,.2],obj_pos,bcolor=np.array([0, 0.7, 0, 1])) #objective
viz.hold(3)

def snag_frames(viz):

    viz.window.setCameraPosition(pos=QtGui.QVector3D(pos[0],pos[1],pos[2]),
                                distance=10,elevation=0,azimuth=90)
    viz.hold(.5)
    with mss.mss() as sct:
        screen_w, screen_h = 1920, 1200
        fraction_of_screen = 0.7 # could be a function parameter
        x = int(screen_w * (1-fraction_of_screen)) // 2
        y = int(screen_h * (1-fraction_of_screen)) // 2
        win_w = int(screen_w * fraction_of_screen)
        win_h = int(screen_h * fraction_of_screen)
        midscreen_w = win_w/2
        midscreen_h = win_h/2
        # monitor = sct.monitors[1]
        monitor_region = {"top": y, "left": x, "width": win_w, "height": win_h}
        scsh = sct.grab(monitor_region)
        sct_image = np.array(scsh)
        # gray_image = cv2.cvtColor(sct_image, cv2.COLOR_BGR2GRAY)
        hsv_image = cv2.cvtColor(sct_image, cv2.COLOR_RGB2HSV)

        obstacle_L_limit = np.array([80,50,50])
        obstacle_U_limit = np.array([100,255,255])
        obstacle_mask = cv2.inRange(hsv_image, obstacle_L_limit, obstacle_U_limit)
        obstacle_image = cv2.bitwise_and(hsv_image, hsv_image, mask=obstacle_mask)
        obstacle_grey = cv2.cvtColor(obstacle_image, cv2.COLOR_BGR2GRAY)
        obs_ret, obs_thresh = cv2.threshold(obstacle_grey, 20, 255, cv2.THRESH_BINARY)
        obs_contours, obs_hierarchy = cv2.findContours(obs_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        obs_x1 = -1*(np.average(obs_contours[0][:,:,0]) - midscreen_w)/117
        obs_z1 = -1*(np.average(obs_contours[0][:,:,1]) - midscreen_h)/117
        # print("Obstacle Centroid 1")
        # print("Obs X: ", obs_x1)
        # print("Obs Z: ", obs_z1)

        objective_L_limit = np.array([55,50,50])
        objective_U_limit = np.array([60,255,255])
        objective_mask = cv2.inRange(hsv_image, objective_L_limit, objective_U_limit)
        objective_image = cv2.bitwise_and(hsv_image, hsv_image, mask=objective_mask)   
        objective_grey = cv2.cvtColor(objective_image, cv2.COLOR_BGR2GRAY)
        obj_ret, obj_thresh = cv2.threshold(objective_grey, 20, 255, cv2.THRESH_BINARY)
        obj_contours, obj_hierarchy = cv2.findContours(obj_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # print("Objective Contours", obj_contours[0])
        obj_x1 = -1*(np.average(obj_contours[0][:,:,0]) - midscreen_w)/117
        obj_z1 = -1*(np.average(obj_contours[0][:,:,1]) - midscreen_h)/117
        # print("Obstacle Centroid 1")
        # print("Obs X: ", obj_x1)
        # print("Obs Z: ", obj_z1)

        # cv2.imshow("Screen", obstacle_image)
        # cv2.waitKey(0)
        # cv2.imshow("Screen", hsv_image)
        # cv2.waitKey(0)
        cv2.destroyAllWindows()

    viz.window.setCameraPosition(pos=QtGui.QVector3D(pos[0],pos[1],pos[2]),
                                distance=10,elevation=0,azimuth=0)
    viz.hold(.5)

    with mss.mss() as sct:
        screen_w, screen_h = 1920, 1200
        fraction_of_screen = 0.7 # could be a function parameter
        x = int(screen_w * (1-fraction_of_screen)) // 2
        y = int(screen_h * (1-fraction_of_screen)) // 2
        win_w = int(screen_w * fraction_of_screen)
        win_h = int(screen_h * fraction_of_screen)
        midscreen_w = win_w/2
        midscreen_h = win_h/2
        # monitor = sct.monitors[1]
        monitor_region = {"top": y, "left": x, "width": win_w, "height": win_h}
        scsh = sct.grab(monitor_region)
        sct_image = np.array(scsh)
        # gray_image = cv2.cvtColor(sct_image, cv2.COLOR_BGR2GRAY)
        hsv_image = cv2.cvtColor(sct_image, cv2.COLOR_RGB2HSV)

        obstacle_L_limit = np.array([80,50,50])
        obstacle_U_limit = np.array([100,255,255])
        obstacle_mask = cv2.inRange(hsv_image, obstacle_L_limit, obstacle_U_limit)
        obstacle_image = cv2.bitwise_and(hsv_image, hsv_image, mask=obstacle_mask)
        obstacle_grey = cv2.cvtColor(obstacle_image, cv2.COLOR_BGR2GRAY)
        obs_ret, obs_thresh = cv2.threshold(obstacle_grey, 20, 255, cv2.THRESH_BINARY)
        obs_contours, obs_hierarchy = cv2.findContours(obs_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        obs_y2 = (np.average(obs_contours[0][:,:,0]) - midscreen_w)/117
        obs_z2 = -1*(np.average(obs_contours[0][:,:,1]) - midscreen_h)/117
        # print("Obstacle Centroid 2")
        # print("Obs Y: ", obs_y2)
        # print("Obs Z: ", obs_z2)

        objective_L_limit = np.array([55,50,50])
        objective_U_limit = np.array([60,255,255])
        objective_mask = cv2.inRange(hsv_image, objective_L_limit, objective_U_limit)
        objective_image = cv2.bitwise_and(hsv_image, hsv_image, mask=objective_mask)   
        objective_grey = cv2.cvtColor(objective_image, cv2.COLOR_BGR2GRAY)
        obj_ret, obj_thresh = cv2.threshold(objective_grey, 20, 255, cv2.THRESH_BINARY)
        obj_contours, obj_hierarchy = cv2.findContours(obj_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # print("Objective Contours", obj_contours[0])
        obj_y2 = (np.average(obj_contours[0][:,:,0]) - midscreen_w)/117
        obj_z2 = -1*(np.average(obj_contours[0][:,:,1]) - midscreen_h)/117
        # print("Obstacle Centroid 2")
        # print("Obs Y: ", obj_y2)
        # print("Obs Z: ", obj_z2)

        # cv2.imshow("Screen", obstacle_image)
        # cv2.waitKey(0)
        # cv2.imshow("Screen", hsv_image)
        # cv2.waitKey(0)
        cv2.destroyAllWindows()

    viz.window.setCameraPosition(pos=QtGui.QVector3D(pos[0],pos[1],pos[2]),
                                distance=10,elevation=90,azimuth=0)
    viz.hold(.5)

    with mss.mss() as sct:
        screen_w, screen_h = 1920, 1200
        fraction_of_screen = 0.7 # could be a function parameter
        x = int(screen_w * (1-fraction_of_screen)) // 2
        y = int(screen_h * (1-fraction_of_screen)) // 2
        win_w = int(screen_w * fraction_of_screen)
        win_h = int(screen_h * fraction_of_screen)
        midscreen_w = win_w/2
        midscreen_h = win_h/2
        # monitor = sct.monitors[1]
        monitor_region = {"top": y, "left": x, "width": win_w, "height": win_h}
        scsh = sct.grab(monitor_region)
        sct_image = np.array(scsh)
        # gray_image = cv2.cvtColor(sct_image, cv2.COLOR_BGR2GRAY)
        hsv_image = cv2.cvtColor(sct_image, cv2.COLOR_RGB2HSV)

        obstacle_L_limit = np.array([80,50,50])
        obstacle_U_limit = np.array([100,255,255])
        obstacle_mask = cv2.inRange(hsv_image, obstacle_L_limit, obstacle_U_limit)
        obstacle_image = cv2.bitwise_and(hsv_image, hsv_image, mask=obstacle_mask)
        obstacle_grey = cv2.cvtColor(obstacle_image, cv2.COLOR_BGR2GRAY)
        obs_ret, obs_thresh = cv2.threshold(obstacle_grey, 20, 255, cv2.THRESH_BINARY)
        obs_contours, obs_hierarchy = cv2.findContours(obs_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        obs_y3 = (np.average(obs_contours[0][:,:,0]) - midscreen_w)/117
        obs_x3 = (np.average(obs_contours[0][:,:,1]) - midscreen_h)/117
        # print("Obstacle Centroid 3")
        # print("Obs X: ", obs_x3)
        # print("Obs Y: ", obs_y3)

        objective_L_limit = np.array([55,50,50])
        objective_U_limit = np.array([60,255,255])
        objective_mask = cv2.inRange(hsv_image, objective_L_limit, objective_U_limit)
        objective_image = cv2.bitwise_and(hsv_image, hsv_image, mask=objective_mask)   
        objective_grey = cv2.cvtColor(objective_image, cv2.COLOR_BGR2GRAY)
        obj_ret, obj_thresh = cv2.threshold(objective_grey, 20, 255, cv2.THRESH_BINARY)
        obj_contours, obj_hierarchy = cv2.findContours(obj_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # print("Objective Contours", obj_contours[0])
        obj_y3 = (np.average(obj_contours[0][:,:,0]) - midscreen_w)/117
        obj_x3 = (np.average(obj_contours[0][:,:,1]) - midscreen_h)/117
        # print("Obstacle Centroid 3")
        # print("Obs X: ", obj_x3)
        # print("Obs Y: ", obj_y3)

        # cv2.imshow("Screen", obstacle_image)
        # cv2.waitKey(0)
        # cv2.imshow("Screen", hsv_image)
        # cv2.waitKey(0)
        cv2.destroyAllWindows()

    obs_output = [[obs_x1, obs_x3],
                  [obs_y2, obs_y3],
                  [obs_z1, obs_z2]]
    obj_output = [[obj_x1, obj_x3],
                 [obj_y2, obj_y3],
                 [obj_z1, obj_z2]]
    
    obs_output = [np.average([obs_x1, obs_x3]),
                  np.average([obs_y2, obs_y3]),
                  np.average([obs_z1, obs_z2])]
    obj_output = [np.average([obj_x1, obj_x3]),
                  np.average([obj_y2, obj_y3]),
                  np.average([obj_z1, obj_z2])]
    
    return obs_output, obj_output

obs_out, obj_out = snag_frames(viz)

for i in range(10):
    ik_out = arm.ik_position(target=obj_out,q0=q_set_0,method="J_T", K = np.eye(3)*.1, kd=.0001,max_iter=10)
    q_set_0 = ik_out[0]
    viz.update(ik_out[0])
    obs_out, obj_out = snag_frames(viz)
    viz.hold(.5)

viz.close_viz()

# print("Obstacle Position: ", str(obs_pos))
# # print("Obstacle X 1: ", obs_x1)
# # print("Obstacle X 3: ", obs_x3)
# # print("Obstacle Y 2: ", obs_y2)
# # print("Obstacle Y 3: ", obs_y3)
# # print("Obstacle Z 1: ", obs_z1)
# # print("Obstacle Z 3: ", obs_z2)
# print(obs_out)

# print("Objective Position: ", str(obj_pos))
# # print("Objective X 1: ", obj_x1)
# # print("Objective X 3: ", obj_x3)
# # print("Objective Y 2: ", obj_y2)
# # print("Objective Y 3: ", obj_y3)
# # print("Objective Z 1: ", obj_z1)
# # print("Objective Z 2: ", obj_z2)
# print(obj_out)